In [1]:
import pandas as pd

In [2]:
version = '01.2022'

## 1. Extract all compound IDs that belong to a Chemical Probes Set

Compound sets per data source:
```
Bromodomains chemical toolbox, 277
Chemical Probes.org, 408
Gray Laboratory Probes, 240
MLP probes, 14 --> Legacy compound set
Open Science Probes, 213
High-quality chemical probes, 395 --> Turn into a boolean label
Nature Chemical Biology Probes, 19 --> Legacy compound set
opnMe Portal, 219
Probe Miner (suitable probes), 227
Protein methyltransferases chemical toolbox, 278
SGC Probes, 28
Tool Compound Set, 19`
Natural product-based probes and drugs, 445
JUMP-Target 1 Compound Set, 443
JUMP-Target 2 Compound Set, 444
Chemical Probes for Understudied Kinases, 409
```


In [64]:
compound2compoundset = pd.read_csv(f"csv_dumps/{version}/compoundtocompoundset.csv", sep=",", header=0, usecols=['compound_id', 'compoundset_id'])
compoundsets = (
    pd.read_csv(
        f"csv_dumps/{version}/compoundset.csv",
        sep=",",
        header=0,
        usecols=['compoundsetid', 'name', 'source_url'])
    .rename(columns={'name' : 'source'})
)

In [100]:
compoundsets[compoundsets.source.str.contains("Natural") | compoundsets.source.str.contains("JUMP") | compoundsets.source.str.contains("Chemical Probes")]

,compoundsetid,source,source_url
55,409,Chemical Probes for Understudied Kinases,https://pubs.acs.org/doi/10.1021/acs.jmedchem....
62,408,Chemical Probes.org,http://new.chemicalprobes.org/
68,443,JUMP-Target 1 Compound Set,https://jump-cellpainting.broadinstitute.org/
72,444,JUMP-Target 2 Compound Set,https://jump-cellpainting.broadinstitute.org/
73,445,Natural product-based probes and drugs,https://pubs.rsc.org/en/content/articlelanding...


In [66]:
probes_compoundsetids = [277, 408, 240, 213, 395, 219, 227, 278, 28, 19, 445, 443, 444, 409]

compound2compoundset = (
    compound2compoundset.merge(
        compoundsets[['compoundsetid', 'source', 'source_url']],
        left_on='compoundset_id',
        right_on='compoundsetid',
        how='inner'
    )
    # Select compounds coming from Probes data sources
    .query('compoundset_id in @probes_compoundsetids')
    .drop(['compoundsetid', 'compoundset_id'], axis=1)
)

print(compound2compoundset.shape[0])
compound2compoundset.head()

5944


,compound_id,source,source_url
13596,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...
13597,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...
13598,32473,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...
13599,7226,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...
13600,26115,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...


# 🧐 Question: If probe in different sources, do we duplicate or want the id to be unique? Unique ID

## 2.1 Pair compound IDs with probes IDs

In [67]:
probes = (
    pd.read_csv(
        f"csv_dumps/{version}/probe.csv",
        sep=",",
        header=0,
        usecols=['probeid', 'compound_id', 'control', 'origin_id'],
        dtype={'probeid':'int64', 'control':'category'})
    .rename(columns={'probeid':'probe_id', 'origin_id':'origin'})
)
probes.head(5)

,probe_id,compound_id,origin,control
0,38210,45800,calculated,NaN
1,37607,21003,calculated,['1']
2,37579,21668,calculated,['1']
3,37581,24527,calculated,['1']
4,36559,45465,calculated,['1']


In [68]:
probes.control.value_counts()

['1']             1037
['BI-6354']          6
['No control']       6
['FM-743']           5
['TP-030n']          5
                  ... 
['BI-1374']          1
['BI-1282']          1
['BI-0900']          1
['BI-0449']          1
['BI-5525']          1
Name: control, Length: 141, dtype: int64

In [69]:
'''
# Controls are read as strings
def str_to_list(X):
    if isinstance(X, str):
        arr = X.strip('[]').split(',')
        arr = [e.strip("''") for e in arr]
        return arr
    return None
    
    
probes['control'] = probes['control'].apply(lambda X: str_to_list(X))

probes = probes.explode('control')
'''

'\n# Controls are read as strings\ndef str_to_list(X):\n    if isinstance(X, str):\n        arr = X.strip(\'[]\').split(\',\')\n        arr = [e.strip("\'\'") for e in arr]\n        return arr\n    return None\n    \n    \nprobes[\'control\'] = probes[\'control\'].apply(lambda X: str_to_list(X))\n\nprobes = probes.explode(\'control\')\n'

In [70]:
# There are 13 compounds not present in the probes dataset
# All coming from 'High-quality chemical probes'

df = compound2compoundset.merge(
    probes,
    on='compound_id',
    how='left'
)

df.head(5)

,compound_id,source,source_url,probe_id,origin,control
0,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses']
1,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,39678.0,experimental,['D-Moses']
2,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35250.0,experimental,['GSK4028']
3,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35459.0,experimental,['GSK4028']
4,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,39684.0,experimental,['GSK4028']


## 2.2 Extract compound name and inchikey

In [71]:
compounds = (
    pd.read_csv(
        f"csv_dumps/{version}/compound.csv",
        sep=",",
        header=0,
        usecols=['compoundid', 'name', 'inchikey'])
    .rename(columns={'compoundid':'compound_id', 'name':'compound_name', 'inchikey':'inchiKey'})
)

In [72]:
df = df.merge(
    compounds,
    on='compound_id',
    how='left'
)

df.head(5)

,compound_id,source,source_url,probe_id,origin,control,compound_name,inchiKey
0,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N
1,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,39678.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N
2,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35250.0,experimental,['GSK4028'],GSK4027,VZAFGXCWAWRULT-UONOGXRCSA-N
3,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35459.0,experimental,['GSK4028'],GSK4027,VZAFGXCWAWRULT-UONOGXRCSA-N
4,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,39684.0,experimental,['GSK4028'],GSK4027,VZAFGXCWAWRULT-UONOGXRCSA-N


## 3. Extract target associated with probe

In [73]:
# First extract target_id
probes2targets = pd.read_csv(f"csv_dumps/{version}/probetobasetarget.csv",
                             sep=",",
                             header=0,
                             usecols=['probe_id', 'basetarget_id'])
                             

df = df.merge(
    probes2targets,
    on='probe_id',
    how='left'
)

In [76]:
print("Counts per source:")

df.groupby('source').count()

Counts per source:


,compound_id,source_url,probe_id,origin,control,compound_name,inchiKey,basetarget_id
source,,,,,,,,
Bromodomains chemical toolbox,177,177,177,177,99,177,177,177
Chemical Probes for Understudied Kinases,94,94,94,94,26,94,94,94
Chemical Probes.org,1363,1363,1363,1363,420,1363,1363,1336
Gray Laboratory Probes,184,184,184,184,0,184,184,180
High-quality chemical probes,1557,1557,1547,1547,515,1557,1557,1508
JUMP-Target 1 Compound Set,393,393,129,129,14,393,393,125
JUMP-Target 2 Compound Set,393,393,129,129,14,393,393,125
Natural product-based probes and drugs,523,523,30,30,2,523,523,30
Nature Chemical Biology Probes,112,112,112,112,3,112,112,73


In [77]:
# Then extract target name

base_targets = (
    pd.read_csv(
        f"csv_dumps/{version}/basetarget.csv",
        sep=",",
        header=0,
        usecols=['basetargetid', 'name', 'gene_name'])
    .rename(columns={'basetargetid':'basetarget_id', 'name':'target_name', 'gene_name':'target_symbols'})
)

In [78]:
df = df.merge(
    base_targets,
    on='basetarget_id',
    how='left'
)

df.head(10)

,compound_id,source,source_url,probe_id,origin,control,compound_name,inchiKey,basetarget_id,target_name,target_symbols
0,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,KAT2B
1,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,40818.0,Histone acetyltransferase KAT2A,KAT2A
2,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,42795.0,Histone acetyltransferase KAT2A/KAT2B,"KAT2A,KAT2B"
3,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,39678.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,KAT2B
4,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,39678.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,40818.0,Histone acetyltransferase KAT2A,KAT2A
5,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,39678.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,42795.0,Histone acetyltransferase KAT2A/KAT2B,"KAT2A,KAT2B"
6,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35250.0,experimental,['GSK4028'],GSK4027,VZAFGXCWAWRULT-UONOGXRCSA-N,33454.0,Histone acetyltransferase KAT2B,KAT2B
7,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35250.0,experimental,['GSK4028'],GSK4027,VZAFGXCWAWRULT-UONOGXRCSA-N,40818.0,Histone acetyltransferase KAT2A,KAT2A
8,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35250.0,experimental,['GSK4028'],GSK4027,VZAFGXCWAWRULT-UONOGXRCSA-N,42795.0,Histone acetyltransferase KAT2A/KAT2B,"KAT2A,KAT2B"
9,29511,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35459.0,experimental,['GSK4028'],GSK4027,VZAFGXCWAWRULT-UONOGXRCSA-N,33454.0,Histone acetyltransferase KAT2B,KAT2B


In [79]:
# Extract uniprot ID from target

basetarget2target = (
    pd.read_csv(
        f"csv_dumps/{version}/targettobasetarget.csv",
        sep=",",
        header=0,
        usecols=['basetarget_id', 'target_id']))

targets = (
    pd.read_csv(
        f"csv_dumps/{version}/target.csv",
        sep=",",
        header=0,
        usecols=['targetid', 'organism_id', 'uniprotid'])
    .rename(columns={'targetid': 'target_id', 'uniprotid':'uniprot_ids'})
)

In [80]:
df = (
    # Map from basetarget to target
    df.merge(
        basetarget2target,
        on='basetarget_id',
        how='left'
    )
    # Extract organism_id and uniprot_ids from target
    .merge(
        targets,
        on='target_id',
        how='left'
    )
    # Select human as organism (organism_id = [5428, 2832])
    .query('organism_id in [5428, 2832]')
)


In [81]:
# Split and explode the uniprot_ids columns

df['uniprot_id'] = df['uniprot_ids'].apply(lambda X: X.split(','))
df = df.explode('uniprot_id')

In [82]:
df.head()

,compound_id,source,source_url,probe_id,origin,control,compound_name,inchiKey,basetarget_id,target_name,target_symbols,target_id,organism_id,uniprot_ids,uniprot_id
0,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,KAT2B,130124.0,2832.0,Q92831,Q92831
1,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,KAT2B,108370.0,2832.0,Q92831,Q92831
2,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,KAT2B,108371.0,2832.0,Q92831,Q92831
3,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,KAT2B,108372.0,2832.0,Q92831,Q92831
5,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,40818.0,Histone acetyltransferase KAT2A,KAT2A,130257.0,2832.0,Q92830,Q92830


## 4. Extract scores

In [83]:
scores = (
    pd.read_csv(
        f"csv_dumps/{version}/compoundtargetscore.csv",
        sep=",",
        header=0,
        usecols=['compound_id', 'basetarget_id', 'percentage', 'score_id'])
    # Filter with only score_ids of interest
    .query('score_id in [1, 5, 6, 7]')
)

df_s = df.merge(
        scores,
        on=['basetarget_id', 'compound_id'],
        how='left'
)

In [84]:
df_s

,compound_id,source,source_url,probe_id,origin,control,compound_name,inchiKey,basetarget_id,target_name,target_symbols,target_id,organism_id,uniprot_ids,uniprot_id,score_id,percentage
0,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,KAT2B,130124.0,2832.0,Q92831,Q92831,7.0,50.0
1,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,KAT2B,108370.0,2832.0,Q92831,Q92831,7.0,50.0
2,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,KAT2B,108371.0,2832.0,Q92831,Q92831,7.0,50.0
3,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,KAT2B,108372.0,2832.0,Q92831,Q92831,7.0,50.0
4,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,40818.0,Histone acetyltransferase KAT2A,KAT2A,130257.0,2832.0,Q92830,Q92830,7.0,30.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
73107,48310,Gray Laboratory Probes,http://graylab.dana-farber.org/probes.html#,35546.0,experimental,NaN,dBRD9,AIOCFZJGGGEWDK-UHFFFAOYSA-N,32700.0,Bromodomain-containing protein 9,BRD9,124444.0,2832.0,Q9H8M2,Q9H8M2,6.0,0.0
73108,48310,Gray Laboratory Probes,http://graylab.dana-farber.org/probes.html#,35546.0,experimental,NaN,dBRD9,AIOCFZJGGGEWDK-UHFFFAOYSA-N,32700.0,Bromodomain-containing protein 9,BRD9,124444.0,2832.0,Q9H8M2,Q9H8M2,7.0,39.6
73109,48310,Gray Laboratory Probes,http://graylab.dana-farber.org/probes.html#,35546.0,experimental,NaN,dBRD9,AIOCFZJGGGEWDK-UHFFFAOYSA-N,32700.0,Bromodomain-containing protein 9,BRD9,124445.0,2832.0,Q9H8M2,Q9H8M2,5.0,92.5
73110,48310,Gray Laboratory Probes,http://graylab.dana-farber.org/probes.html#,35546.0,experimental,NaN,dBRD9,AIOCFZJGGGEWDK-UHFFFAOYSA-N,32700.0,Bromodomain-containing protein 9,BRD9,124445.0,2832.0,Q9H8M2,Q9H8M2,6.0,0.0


In [85]:
df_s["probesDrugsScore"] = df_s.apply(lambda X: X.percentage if X.score_id == 7 else None, axis=1)
df_s['probeMinerScore'] = df_s.apply(lambda X: X.percentage if X.score_id == 1 else None, axis=1)
df_s['scoreInCells'] = df_s.apply(lambda X: X.percentage if X.score_id == 5 else None, axis=1)
df_s['scoreInOrganisms'] = df_s.apply(lambda X: X.percentage if X.score_id == 6 else None, axis=1)

In [86]:
# Right now I have duplicated rows due to:
# several data sources evidencing the same assoc +
# exploded uniprot ids +
# I have to coalesce the score rows

In [87]:
df = df_s.copy()

## 5. Extract MoA

In [88]:
moas = (
    pd.read_csv(
        f"csv_dumps/{version}/compoundaction.csv",
        sep=",",
        header=0,
        usecols=['compound_id', 'target_id', 'actiontype_id'])
)

In [89]:
df = df.merge(
        moas,
        on=['target_id', 'compound_id'],
        how='left'
)

In [90]:
df.head()

,compound_id,source,source_url,probe_id,origin,control,compound_name,inchiKey,basetarget_id,target_name,...,organism_id,uniprot_ids,uniprot_id,score_id,percentage,probesDrugsScore,probeMinerScore,scoreInCells,scoreInOrganisms,actiontype_id
0,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,...,2832.0,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN
1,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,...,2832.0,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN
2,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,...,2832.0,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN
3,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,...,2832.0,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN
4,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,40818.0,Histone acetyltransferase KAT2A,...,2832.0,Q92830,Q92830,7.0,30.0,30.0,NaN,NaN,NaN,NaN


## 6. Map to drugId

In [91]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = (SparkSession.builder
    .appName('spark')
    .getOrCreate())

In [92]:

drug_idx = (
    spark.read.parquet('/Users/irene/Documents/dev/pyspark/22.04.1/molecule')
    .select(col('id').alias('drugId'), 'inchiKey')
    .toPandas()
)

In [93]:
df = (
    # Get drugId
    df.merge(
        drug_idx,
        on='inchiKey',
        how='left')
)


df.head()

,compound_id,source,source_url,probe_id,origin,control,compound_name,inchiKey,basetarget_id,target_name,...,uniprot_ids,uniprot_id,score_id,percentage,probesDrugsScore,probeMinerScore,scoreInCells,scoreInOrganisms,actiontype_id,drugId
0,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,...,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN,NaN
1,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,...,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN,NaN
2,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,...,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN,NaN
3,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,33454.0,Histone acetyltransferase KAT2B,...,Q92831,Q92831,7.0,50.0,50.0,NaN,NaN,NaN,NaN,NaN
4,30108,Bromodomains chemical toolbox,https://www.nature.com/articles/s41467-019-096...,35251.0,experimental,['D-Moses'],L-Moses,MSFPLTWUFWOKBX-IFXJQAMLSA-N,40818.0,Histone acetyltransferase KAT2A,...,Q92830,Q92830,7.0,30.0,30.0,NaN,NaN,NaN,NaN,NaN


In [94]:
df = df.astype({
    'probe_id': 'Int64',
    'basetarget_id': 'Int64',
    'target_id': 'Int64',
    'organism_id': 'Int64',
    'score_id': 'Int64'
})

In [96]:
df.to_csv(f'output/{version}/chemprobes_{version}_1.tsv', sep='\t', header=True, index=False)

In [95]:
df.iloc[0]

compound_id                                                     30108
source                                  Bromodomains chemical toolbox
source_url          https://www.nature.com/articles/s41467-019-096...
probe_id                                                        35251
origin                                                   experimental
control                                                   ['D-Moses']
compound_name                                                 L-Moses
inchiKey                                  MSFPLTWUFWOKBX-IFXJQAMLSA-N
basetarget_id                                                   33454
target_name                           Histone acetyltransferase KAT2B
target_symbols                                                  KAT2B
target_id                                                      130124
organism_id                                                      2832
uniprot_ids                                                    Q92831
uniprot_id          

## 7. Build final dataset

In [101]:
data = spark.read.csv(f'output/{version}/chemprobes_{version}_1.tsv', sep='\t', header=True)

All posible URLs
```
+-------------------------------------------+---------------------------------------------------------------+
|source                                     |source_url                                                     |
+-------------------------------------------+---------------------------------------------------------------+
|Probe Miner (suitable probes)              |http://probeminer.icr.ac.uk/#/                                 |
|High-quality chemical probes               |null                                                           |
|Chemical Probes.org                        |http://new.chemicalprobes.org/                                 |
|opnMe Portal                               |https://opnme.com/                                             |
|Bromodomains chemical toolbox              |https://www.nature.com/articles/s41467-019-09672-2#article-info|
|SGC Probes                                 |http://www.thesgc.org/chemical-probes                          |
|Gray Laboratory Probes                     |http://graylab.dana-farber.org/probes.html#                    |
|Protein methyltransferases chemical toolbox|https://www.nature.com/articles/s41467-018-07905-4             |
|Nature Chemical Biology Probes             |http://www.nature.com/nchembio/chemical_probes.html            |
|Open Science Probes                        |http://www.sgc-ffm.uni-frankfurt.de/#!start
|Natural product-based probes and drugs     |https://pubs.rsc.org/en/content/articlelanding/2022/NP/D1NP00039J
|JUMP-Target 1 Compound Set                 |https://jump-cellpainting.broadinstitute.org/                  |
|JUMP-Target 2 Compound Set                 |https://jump-cellpainting.broadinstitute.org/                  |
|Chemical Probes for Understudied Kinases   |https://pubs.acs.org/doi/10.1021/acs.jmedchem.1c00980          |
```


In [126]:
data.first()

Row(compound_id='30108', source='Bromodomains chemical toolbox', source_url='https://www.nature.com/articles/s41467-019-09672-2#article-info', probe_id='35251', origin='experimental', control="['D-Moses']", compound_name='L-Moses', inchiKey='MSFPLTWUFWOKBX-IFXJQAMLSA-N', basetarget_id='33454', target_name='Histone acetyltransferase KAT2B', target_symbols='KAT2B', target_id='130124', organism_id='2832', uniprot_ids='Q92831', uniprot_id='Q92831', score_id='7', percentage='50.0', probesDrugsScore='50.0', probeMinerScore=None, scoreInCells=None, scoreInOrganisms=None, actiontype_id=None, drugId=None)

In [104]:
(data
.filter(col('compound_id') == '888')
).show(10, False, True)

-RECORD 0-----------------------------------------------------------------------------------------
 compound_id      | 888                                                                           
 source           | High-quality chemical probes                                                  
 source_url       | https://pubs.rsc.org/en/content/articlelanding/2021/md/d1md00138h             
 probe_id         | 39871                                                                         
 origin           | calculated                                                                    
 control          | null                                                                          
 compound_name    | TGX-221                                                                       
 inchiKey         | CPRAGQJXBLMUEL-UHFFFAOYSA-N                                                   
 basetarget_id    | 35770                                                                         
 target_na

In [121]:
data_m = (data
    # Drop columns that are not needed
    .drop('compound_id', 'basetarget_id', 'target_id', 'organism_id', 'score_id', 'percentage', 'uniprot_ids', 'probe_id')
    # Change Probe Miner source name
    .withColumn('source', when(col('source').contains('Probe Miner'), 'Probe Miner').otherwise(col('source')))
    # Build more complete URLs
    .withColumn(
    'source_url',
    when(col('source_url').contains('probeminer'), concat('source_url', 'uniprot_id'))
    .when(col('source_url').contains('probes.org'), concat(lit('https://new.chemicalprobes.org/?q='), 'compound_name'))
    .when(col('source_url').contains('thesgc'), concat('source_url', lit('/'), 'compound_name'))
    .when(col('source_url').contains('graylab'), lit(None))
    .when(col('source_url').contains('opnme'), concat(lit('https://opnme.com/search-results/'), 'compound_name'))
    .when(col('source_url').contains('frankfurt'),
          concat(
              element_at(split(col('source_url'), '#!start'), 1),
              lit('#!specificprobeoverview/'),
              'compound_name'))
    .otherwise(col('source_url')))
    # Create high-quality flag
    .withColumn('isHighQuality', when(col('source') == 'High-quality chemical probes', True).otherwise(False))
    # Clean rows where control = 1
    .withColumn('control', when(~col('control').contains(1), col('control')))
    .withColumn('control', when(~col('control').contains('2S'), col('control')))
    # Group data and collect sets
    .groupBy('uniprot_id', 'compound_name', 'drugId', 'inchiKey')
    .agg(
        collect_set(
            struct(
                col('source').alias('niceName'),
                col('source_url').alias('url'))
        ).alias('urls'),
        flatten(collect_set(array("control"))).alias('control'), # max size = 1
        collect_set("probesDrugsScore").alias('probesDrugsScore'),
        collect_set("probeMinerScore").alias('probeMinerScore'),
        collect_set("scoreInCells").alias('scoreInCells'),
        collect_set("scoreInOrganisms").alias('scoreInOrganisms'),
        flatten(collect_set(array("actiontype_id"))).alias('mechanismOfAction'),
        flatten(collect_set(array("isHighQuality"))).alias('isHighQuality'),
        collect_set("origin").alias("origin")
    )
    .withColumn('probesDrugsScore', array_max(col('probesDrugsScore')))
    .withColumn('probeMinerScore', array_max(col('probeMinerScore')))
    .withColumn('scoreInCells', array_max(col('scoreInCells')))
    .withColumn('scoreInOrganisms', array_max(col('scoreInOrganisms')))
    .withColumn('mechanismOfAction', expr('filter(mechanismOfAction, x -> x is not null)'))
    .withColumn('control', explode(col('control')))
    .withColumnRenamed('compound_name', 'id')
    .withColumnRenamed('uniprot_id', 'targetFromSourceId')
    .withColumn('urls', array_distinct(col('urls')))
    # Remove reference to High Quality Probes source as this is a subset curated by P&Ds
    # Problem: there are compounds only referenced with High Quality Probes 
    # Solution: remove it if there is already a reference, otherwise reference P&Ds 
    .withColumn('urls', 
        array_except(col('urls'), array(struct(lit('High-quality chemical probes').alias("niceName"),
        lit(None).cast("string").alias("url")))))
    .withColumn( 
        'urls',
        when(
            size(col('urls')) == 0,
            array(struct(
                lit('Probes & Drugs Portal').alias('niceName'),
                lit('https://www.probes-drugs.org').alias('url')))
        )
        .otherwise(col('urls'))
    )
    # Convert empty arrays into null
    .withColumn('mechanismOfAction', when(size(col('mechanismOfAction')) == 0, lit(None)).otherwise(col('mechanismOfAction')))
    .withColumn('origin', when(size(col('origin')) == 0, lit(None)).otherwise(col('origin')))
    # Transform isHighQuality to keep only True
    .withColumn('isHighQuality', when(array_contains(col('isHighQuality'), True), True).otherwise(False))
    .distinct()
)

data_m.count()

4320

In [122]:
data_m.first()

Row(targetFromSourceId='B2RXH2', id='IOX1', drugId=None, inchiKey='JGRPKOGHYBAVMW-UHFFFAOYSA-N', urls=[Row(niceName='High-quality chemical probes', url='https://pubs.rsc.org/en/content/articlelanding/2021/md/d1md00138h'), Row(niceName='SGC Probes', url='http://www.thesgc.org/chemical-probes/IOX1')], control=None, probesDrugsScore='20.4', probeMinerScore='44.0', scoreInCells=None, scoreInOrganisms=None, mechanismOfAction=None, isHighQuality=True, origin=['experimental'])

In [131]:
data.withColumn('source_and_url', concat_ws('|', col('source'), col('source_url'))).first()

Row(compound_id='30108', source='Bromodomains chemical toolbox', source_url='https://www.nature.com/articles/s41467-019-09672-2#article-info', probe_id='35251', origin='experimental', control="['D-Moses']", compound_name='L-Moses', inchiKey='MSFPLTWUFWOKBX-IFXJQAMLSA-N', basetarget_id='33454', target_name='Histone acetyltransferase KAT2B', target_symbols='KAT2B', target_id='130124', organism_id='2832', uniprot_ids='Q92831', uniprot_id='Q92831', score_id='7', percentage='50.0', probesDrugsScore='50.0', probeMinerScore=None, scoreInCells=None, scoreInOrganisms=None, actiontype_id=None, drugId=None, source_and_url='Bromodomains chemical toolbox|https://www.nature.com/articles/s41467-019-09672-2#article-info')

In [132]:
test = (data
    # Drop columns that are not needed
    .drop('compound_id', 'basetarget_id', 'target_id', 'organism_id', 'score_id', 'percentage', 'uniprot_ids', 'probe_id')
    # Change Probe Miner source name
    .withColumn('source', when(col('source').contains('Probe Miner'), 'Probe Miner').otherwise(col('source')))
    # Build more complete URLs
    .withColumn(
    'source_url',
    when(col('source_url').contains('probeminer'), concat('source_url', 'uniprot_id'))
    .when(col('source_url').contains('probes.org'), concat(lit('https://new.chemicalprobes.org/?q='), 'compound_name'))
    .when(col('source_url').contains('thesgc'), concat('source_url', lit('/'), 'compound_name'))
    .when(col('source_url').contains('graylab'), lit(None))
    .when(col('source_url').contains('opnme'), concat(lit('https://opnme.com/search-results/'), 'compound_name'))
    .when(col('source_url').contains('frankfurt'),
          concat(
              element_at(split(col('source_url'), '#!start'), 1),
              lit('#!specificprobeoverview/'),
              'compound_name'))
    .otherwise(col('source_url')))
    # Create high-quality flag and set source to null if it is High Quality Probes
    .withColumn('isHighQuality', when(col('source') == 'High-quality chemical probes', True).otherwise(False))
    .withColumn('source', when(col('source').contains('High-quality chemical probes'), lit(None)).otherwise(col('source')))
    # Zip source and url in the same string not to lose their correspondence
    .withColumn('source_and_url', concat_ws('|', col('source'), col('source_url')))
    # Clean rows where control = 1
    .withColumn('control', when(~col('control').contains(1), col('control')))
    .withColumn('control', when(~col('control').contains('2S'), col('control')))
    # Group data and collect sets
    .groupBy('uniprot_id', 'compound_name', 'drugId', 'inchiKey')
    .agg(
        collect_set(col('source_and_url')).alias('urls'),
        flatten(collect_set(array("control"))).alias('control'), # max size = 1
        collect_set("probesDrugsScore").alias('probesDrugsScore'),
        collect_set("probeMinerScore").alias('probeMinerScore'),
        collect_set("scoreInCells").alias('scoreInCells'),
        collect_set("scoreInOrganisms").alias('scoreInOrganisms'),
        flatten(collect_set(array("actiontype_id"))).alias('mechanismOfAction'),
        flatten(collect_set(array("isHighQuality"))).alias('isHighQuality'),
        collect_set("origin").alias("origin")
    )
    .withColumn('probesDrugsScore', array_max(col('probesDrugsScore')))
    .withColumn('probeMinerScore', array_max(col('probeMinerScore')))
    .withColumn('scoreInCells', array_max(col('scoreInCells')))
    .withColumn('scoreInOrganisms', array_max(col('scoreInOrganisms')))
    .withColumn('mechanismOfAction', expr('filter(mechanismOfAction, x -> x is not null)'))
    .withColumn('control', explode(col('control')))
    .withColumnRenamed('compound_name', 'id')
    .withColumnRenamed('uniprot_id', 'targetFromSourceId')
    # Remove reference to High Quality Probes source as this is a subset curated by P&Ds
    # Problem: there are compounds only referenced with High Quality Probes 
    # Solution: remove it if there is already a reference, otherwise reference P&Ds 
    # Convert empty arrays into null
    .withColumn('mechanismOfAction', when(size(col('mechanismOfAction')) == 0, lit(None)).otherwise(col('mechanismOfAction')))
    .withColumn('origin', when(size(col('origin')) == 0, lit(None)).otherwise(col('origin')))
    # Transform isHighQuality to keep only True
    .withColumn('isHighQuality', when(array_contains(col('isHighQuality'), True), True).otherwise(False))
    .distinct()
)

test.show(truncate=False)

+------------------+---------+------+---------------------------+--------------------------------------------------------------------------------------------------------------------------+-------+----------------+---------------+------------+----------------+-----------------+-------------+--------------+
|targetFromSourceId|id       |drugId|inchiKey                   |urls                                                                                                                      |control|probesDrugsScore|probeMinerScore|scoreInCells|scoreInOrganisms|mechanismOfAction|isHighQuality|origin        |
+------------------+---------+------+---------------------------+--------------------------------------------------------------------------------------------------------------------------+-------+----------------+---------------+------------+----------------+-----------------+-------------+--------------+
|B2RXH2            |IOX1     |null  |JGRPKOGHYBAVMW-UHFFFAOYSA-N|[SGC Probes|ht

In [134]:
test.filter(size(col('urls')) > 1).first()

Row(targetFromSourceId='B2RXH2', id='IOX1', drugId=None, inchiKey='JGRPKOGHYBAVMW-UHFFFAOYSA-N', urls=['SGC Probes|http://www.thesgc.org/chemical-probes/IOX1', 'https://pubs.rsc.org/en/content/articlelanding/2021/md/d1md00138h'], control=None, probesDrugsScore='20.4', probeMinerScore='44.0', scoreInCells=None, scoreInOrganisms=None, mechanismOfAction=None, isHighQuality=True, origin=['experimental'])

In [55]:
data_m.show(10, False, True)

-RECORD 0---------------------------------------------------------------------------------------------------------------------------------------------------------------------------
 targetFromSourceId | B2RXH2                                                                                                                                                        
 id                 | IOX1                                                                                                                                                          
 drugId             | null                                                                                                                                                          
 inchiKey           | JGRPKOGHYBAVMW-UHFFFAOYSA-N                                                                                                                                   
 urls               | [{High-quality chemical probes, https://pubs.rsc.org/en/content/articlela

In [136]:
data_m.withColumn('s', explode('urls')).groupBy('s.niceName').count().toPandas()

,niceName,count
0,High-quality chemical probes,940
1,Open Science Probes,130
2,JUMP-Target 2 Compound Set,72
3,Probe Miner,3255
4,Chemical Probes.org,748
5,opnMe Portal,87
6,Chemical Probes for Understudied Kinases,41
7,Bromodomains chemical toolbox,55
8,Gray Laboratory Probes,134
9,Protein methyltransferases chemical toolbox,28


In [56]:
data_m.coalesce(1).write.json(f'output/{version}/chemicalprobes_{version}', mode='overwrite')